In [116]:
#Lab5 Ashwini Balachandra
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
import sqlite3
from sqlite3 import Error
people_db_file = "sqlite.db"# The name of the database file to use
max_people = 500 # Number  of records to create

In [117]:
#Part1
def generate_people(count):
    # with open('LastNames.txt', 'r') as filehandle:
    #    last_names = [line.rstrip()  for line in filehandle.readlines()]
    # with open('FirstNames.txt', 'r') as filehandle1:
    #    first_names = [line.rstrip() for line in filehandle1.readlines()]
    last_names = []
    first_names =[]
    with ThreadPoolExecutor(max_workers=2) as executor:
         future = (executor.submit(txt_list,'LastNames.txt'))
         future1 = (executor.submit(txt_list,'FirstNames.txt'))
         last_names = future.result()
         first_names = future1.result()
              
    my_tuple = ()
    names = [(counter,first_names[random.randint(0,len(first_names)-1)],last_names[random.randint(0,len(last_names)-1)]) for counter in range(count)]
    return names
                   
def txt_list(file_name):
    with open(file_name, 'r') as filehandle:
         lt_names = [line.rstrip()  for line in filehandle.readlines()]  
    return lt_names
                   
                   

In [118]:
print(generate_people(5))

[(0, 'STELLA', 'SPARKS'), (1, 'ZULMA', 'SMALL'), (2, 'DARNELL', 'SNOW'), (3, 'KRAIG', 'DUKE'), (4, 'GILBERTO', 'HUMPHREY')]


In [119]:
if __name__ == "__main__":
    people = generate_people(5)
    print(people)
   

[(0, 'TAMEKIA', 'PATEL'), (1, 'STEFANIA', 'LITTLE'), (2, 'LATASHIA', 'VALENCIA'), (3, 'EARNEST', 'MCDANIEL'), (4, 'CHERYLL', 'CAMPBELL')]


In [120]:
#Part2
def create_people_database(db_file, count):
    conn = sqlite3.connect(db_file)
    with conn:
        
        sql_create_people_table = """ CREATE TABLE IF NOT EXISTS people (
        id integer PRIMARY KEY,
        first_name text NOT NULL,last_name text NOT NULL); """
        cursor = conn.cursor()
        cursor.execute(sql_create_people_table)
        sql_truncate_people = "DELETE FROM people;"
        cursor.execute(sql_truncate_people)
        people = generate_people(count)
        sql_insert_person = "INSERT INTO people(id,first_name,last_name) VALUES(?,?,?);"
        for person in people:
            print(person)# uncomment if you want to see the person object
            cursor.execute(sql_insert_person, person)
            print(cursor.lastrowid)# uncomment if you want to see the row id 
        cursor.close()

In [121]:
if __name__ == "__main__":
    people = generate_people(5)
    print(people)
    create_people_database(people_db_file, 400)

[(0, 'ZACHARY', 'HERNANDEZ'), (1, 'MAURO', 'HUBER'), (2, 'ARTHUR', 'SPENCE'), (3, 'SHAYLA', 'SHEPHERD'), (4, 'MAUREEN', 'ROSALES')]
(0, 'LEEANN', 'CISNEROS')
0
(1, 'QUEEN', 'SHORT')
1
(2, 'ALETA', 'CHAMBERS')
2
(3, 'GABRIELLE', 'PEREZ')
3
(4, 'TAYLOR', 'LANDRY')
4
(5, 'HYON', 'SALAZAR')
5
(6, 'CHRISTEEN', 'CONRAD')
6
(7, 'JOLENE', 'RANGEL')
7
(8, 'ESMERALDA', 'HANSEN')
8
(9, 'ANH', 'PAUL')
9
(10, 'SOOK', 'MCKAY')
10
(11, 'COLUMBUS', 'BOND')
11
(12, 'MILISSA', 'MCCARTY')
12
(13, 'THEDA', 'MIRANDA')
13
(14, 'VERNELL', 'GREEN')
14
(15, 'MYRA', 'STEIN')
15
(16, 'BORIS', 'WARD')
16
(17, 'BART', 'HARDY')
17
(18, 'HENRY', 'FREEMAN')
18
(19, 'ABRAHAM', 'TANNER')
19
(20, 'MARTINA', 'HORNE')
20
(21, 'ELKE', 'MARKS')
21
(22, 'AIDA', 'MCFARLAND')
22
(23, 'KENNETH', 'HOFFMAN')
23
(24, 'DEANN', 'GALLOWAY')
24
(25, 'SHELLIE', 'BOND')
25
(26, 'RACHELLE', 'CLAYTON')
26
(27, 'CHAD', 'HOLLAND')
27
(28, 'TESSIE', 'GALLEGOS')
28
(29, 'CONNIE', 'MERCADO')
29
(30, 'PARTICIA', 'CRANE')
30
(31, 'SCOTT', 'ROGER

In [122]:
#Part 3
class PersonDB():
    def __init__(self, db_file=''):
        ''' Store db_file parameter value '''
        self.db_file = db_file

    def __enter__(self):
        ''' Initiate connection to database '''
        self.conn = sqlite3.connect(self.db_file,check_same_thread=False)
        return self

    def __exit__(self, exc_type, exc_value, exc_traceback):
        ''' Close database connection '''
        self.conn.close()

    def load_person(self, id):
        sql = "SELECT * FROM people WHERE id=?"

        cursor = self.conn.cursor()
        cursor.execute(sql, (id,))
        records = cursor.fetchall()
        result = (-1,'','') # id = -1, first_name = '', last_name = ''

        if records is not None and len(records) > 0:
                result = records[0]
        cursor.close()
        return result

In [123]:
def test_PersonDB():
    with PersonDB(people_db_file) as db:
        print(db.load_person(10000)) # Should print the default
        print(db.load_person(122))
        print(db.load_person(300))
        
print(test_PersonDB())

(-1, '', '')
(122, 'MITCHEL', 'STANLEY')
(300, 'ELIZEBETH', 'DODSON')
None


In [134]:
#PART 4

def load_person(id, db_file):
    with PersonDB(db_file) as db:
        return db.load_person(id)
    
    
    
def load_all_people():
    worker_thread = 10
    result_list = []
    with ThreadPoolExecutor(worker_thread ) as executor:
        futures = [executor.submit(load_person, x,people_db_file) for x in range(max_people -1)]
        for future in as_completed(futures):
            result_list.append(future.result())
            #print(future.result())
        print(result_list)
        result_list.sort(key=lambda x:(x[2],x[1]))
        print(result_list)

if __name__ == "__main__":
    #people = generate_people(300)
    #print(people)
    create_people_database(people_db_file, max_people-1)
    load_all_people()
    


(0, 'FATIMA', 'MAYER')
0
(1, 'EMMY', 'WALLS')
1
(2, 'KASSANDRA', 'DOMINGUEZ')
2
(3, 'FRANKLYN', 'GRIFFITH')
3
(4, 'BLAKE', 'COCHRAN')
4
(5, 'MONTY', 'SKINNER')
5
(6, 'TINA', 'CLAYTON')
6
(7, 'JESUS', 'ATKINS')
7
(8, 'RAY', 'COLEMAN')
8
(9, 'ERIC', 'MCMAHON')
9
(10, 'CHRISTINE', 'RICE')
10
(11, 'ARTURO', 'AVILA')
11
(12, 'SHANNON', 'DAVID')
12
(13, 'CHARLOTTE', 'CAREY')
13
(14, 'LEANNE', 'HERMAN')
14
(15, 'LYMAN', 'VALENTINE')
15
(16, 'VIKI', 'MAXWELL')
16
(17, 'CRISSY', 'KENT')
17
(18, 'AUTUMN', 'VALDEZ')
18
(19, 'TEODORO', 'COLLINS')
19
(20, 'LAMONT', 'SHORT')
20
(21, 'MAGALY', 'SILVA')
21
(22, 'CANDI', 'SEXTON')
22
(23, 'DANIELLE', 'LOWERY')
23
(24, 'HUMBERTO', 'JENNINGS')
24
(25, 'JOHN', 'VALENZUELA')
25
(26, 'VERNON', 'SHORT')
26
(27, 'GAYNELLE', 'TODD')
27
(28, 'MONA', 'DAVIES')
28
(29, 'EDDIE', 'MANNING')
29
(30, 'LANELL', 'CAMACHO')
30
(31, 'ROSLYN', 'SCHWARTZ')
31
(32, 'TOMI', 'HEATH')
32
(33, 'HUGH', 'WINTERS')
33
(34, 'ESMERALDA', 'YOUNG')
34
(35, 'JUDI', 'GARRETT')
35
(36, '

[(1, 'EMMY', 'WALLS'), (10, 'CHRISTINE', 'RICE'), (12, 'SHANNON', 'DAVID'), (13, 'CHARLOTTE', 'CAREY'), (8, 'RAY', 'COLEMAN'), (16, 'VIKI', 'MAXWELL'), (7, 'JESUS', 'ATKINS'), (9, 'ERIC', 'MCMAHON'), (18, 'AUTUMN', 'VALDEZ'), (14, 'LEANNE', 'HERMAN'), (0, 'FATIMA', 'MAYER'), (6, 'TINA', 'CLAYTON'), (2, 'KASSANDRA', 'DOMINGUEZ'), (3, 'FRANKLYN', 'GRIFFITH'), (4, 'BLAKE', 'COCHRAN'), (15, 'LYMAN', 'VALENTINE'), (5, 'MONTY', 'SKINNER'), (11, 'ARTURO', 'AVILA'), (17, 'CRISSY', 'KENT'), (19, 'TEODORO', 'COLLINS'), (20, 'LAMONT', 'SHORT'), (23, 'DANIELLE', 'LOWERY'), (25, 'JOHN', 'VALENZUELA'), (22, 'CANDI', 'SEXTON'), (30, 'LANELL', 'CAMACHO'), (24, 'HUMBERTO', 'JENNINGS'), (27, 'GAYNELLE', 'TODD'), (28, 'MONA', 'DAVIES'), (21, 'MAGALY', 'SILVA'), (32, 'TOMI', 'HEATH'), (26, 'VERNON', 'SHORT'), (35, 'JUDI', 'GARRETT'), (31, 'ROSLYN', 'SCHWARTZ'), (37, 'AURA', 'GOODWIN'), (33, 'HUGH', 'WINTERS'), (29, 'EDDIE', 'MANNING'), (36, 'JOSEPH', 'BLACKBURN'), (38, 'JOAN', 'COCHRAN'), (46, 'JENNINE', 